In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

# Life Hack 2023

In [7]:
data = pd.read_csv(
    './submission_data.csv',
    low_memory=False
)

In [13]:
data['launch_date'] = pd.to_datetime(data['launch_date'], errors='coerce')

launch_2023 = data[data['launch_date'].dt.year == 2023]
launch_2023.head()

,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,ind_launch_date,indication,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target
20,BRAND_056A,1.780228,2.004200,BRAND_056A_COUNTRY_0309,CORP_01C7,COUNTRY_0309,2023-01-01,2023-01-01,DRUG_ID_EC3A,2022-09-01 00:00:00,['IND_BAFB'],2.026286,1.498170,0.001062,1.280739,1.401970,1.885995,THER_AREA_96D7,NaN
78,BRAND_9259,1.130150,1.371908,BRAND_9259_COUNTRY_3536,CORP_01C7,COUNTRY_3536,2023-01-01,2023-01-01,DRUG_ID_1315,2022-02-01 00:00:00,['IND_BAFB'],1.480000,1.339915,0.000234,1.180618,1.046781,1.731343,THER_AREA_6CEE,NaN
161,BRAND_C352,1.004234,1.068500,BRAND_C352_COUNTRY_F78A,CORP_01C7,COUNTRY_F78A,2023-02-01,2023-02-01,DRUG_ID_0C4E,2021-10-01 00:00:00,['IND_851C'],-1.000000,2.380330,0.018129,-1.000000,1.067755,1.016166,THER_AREA_CD59,NaN
174,BRAND_D786,1.046707,1.274527,BRAND_D786_COUNTRY_1033,CORP_01C7,COUNTRY_1033,2023-02-01,2023-02-01,DRUG_ID_C48E,2022-08-01 00:00:00,['IND_B2EF'],1.133333,1.556702,0.001079,-1.000000,1.285974,1.791045,THER_AREA_96D7,NaN
194,BRAND_9259,1.130150,1.371908,BRAND_9259_COUNTRY_3536,CORP_01C7,COUNTRY_3536,2023-01-01,2023-02-01,DRUG_ID_1315,2022-02-01 00:00:00,['IND_BAFB'],1.480000,1.339915,0.000234,1.180618,1.051592,1.731343,THER_AREA_6CEE,NaN


In [11]:
# Grouping the data by 'country' and 'therapeutic_area'
country_therapy_brands = (
    launch_2023.groupby(['country', 'therapeutic_area'])['brand']
    .unique()
    .reset_index()
    .rename(columns={'brand': 'brands'})
)
country_therapy_brands.head()


,country,therapeutic_area,brands
0,COUNTRY_0309,THER_AREA_96D7,"[BRAND_056A, BRAND_D786, BRAND_B8DE]"
1,COUNTRY_06E4,THER_AREA_6CEE,[BRAND_9259]
2,COUNTRY_06E4,THER_AREA_96D7,[BRAND_056A]
3,COUNTRY_0C7D,THER_AREA_6CEE,[BRAND_9259]
4,COUNTRY_0C7D,THER_AREA_CD59,[BRAND_C352]


In [14]:
len(country_therapy_brands)

56

In [17]:
data2 = pd.read_csv(
    './train_data.csv',
    low_memory=False
)

data2.head()

,brand,che_pc_usd,che_perc_gdp,cluster_nl,corporation,country,launch_date,date,drug_id,ind_launch_date,indication,insurance_perc_che,population,prev_perc,price_month,price_unit,public_perc_che,therapeutic_area,target
0,BRAND_354E,1.209114,1.665879,BRAND_354E_COUNTRY_88A3,CORP_D524,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_8795,-1,['IND_C3B6'],1.893333,1.008039,0.028367,1.006444,1.013784,1.835821,THER_AREA_980E,1.000784
1,BRAND_626D,-1.000000,-1.000000,BRAND_626D_COUNTRY_8B47,CORP_01C7,COUNTRY_8B47,2014-06-01,2014-06-01,DRUG_ID_E66E,2014-09-01 00:00:00,"['IND_1590', 'IND_ECAC']",-1.000000,1.023562,0.000047,-1.000000,1.626677,-1.000000,THER_AREA_96D7,1.000000
2,BRAND_45D9,1.209114,1.665879,BRAND_45D9_COUNTRY_88A3,CORP_39F7,COUNTRY_88A3,2014-06-01,2014-06-01,DRUG_ID_F272,-1,['IND_B2EF'],1.893333,1.008039,0.001502,-1.000000,3.144874,1.835821,THER_AREA_96D7,1.002258
3,BRAND_D724,1.851280,2.051770,BRAND_D724_COUNTRY_445D,CORP_711A,COUNTRY_445D,2014-06-01,2014-06-01,DRUG_ID_1D4E,-1,['IND_BAFB'],1.000000,1.253186,0.001304,-1.000000,1.213446,1.805970,THER_AREA_6CEE,1.068761
4,BRAND_4887,1.791199,2.059130,BRAND_4887_COUNTRY_D8B0,CORP_443D,COUNTRY_D8B0,2014-06-01,2014-06-01,DRUG_ID_AA88,-1,['IND_3F31'],2.013333,1.639352,0.054467,1.018589,1.008708,1.880597,THER_AREA_6CEE,1.036312


In [19]:

train_file_path = './train_data.csv'
train_data = pd.read_csv(train_file_path)

train_data['launch_date'] = pd.to_datetime(train_data['launch_date'], errors='coerce')

price_unit_stats = (
    train_data.groupby(['country', 'therapeutic_area'])['price_unit']
    .agg(['mean', 'median', 'std', 'min', 'max', 'count'])
    .reset_index()
)

merged_data = pd.merge(
    country_therapy_brands,
    price_unit_stats,
    on=['country', 'therapeutic_area'],
    how='left'
)

# Display the merged data with price_unit statistics
print(merged_data)

         country therapeutic_area                                brands  \
0   COUNTRY_0309   THER_AREA_96D7  [BRAND_056A, BRAND_D786, BRAND_B8DE]   
1   COUNTRY_06E4   THER_AREA_6CEE                          [BRAND_9259]   
2   COUNTRY_06E4   THER_AREA_96D7                          [BRAND_056A]   
3   COUNTRY_0C7D   THER_AREA_6CEE                          [BRAND_9259]   
4   COUNTRY_0C7D   THER_AREA_CD59                          [BRAND_C352]   
5   COUNTRY_1007   THER_AREA_644A                          [BRAND_DB5A]   
6   COUNTRY_1007   THER_AREA_96D7                          [BRAND_056A]   
7   COUNTRY_1007   THER_AREA_980E                          [BRAND_E57A]   
8   COUNTRY_1033   THER_AREA_96D7                          [BRAND_D786]   
9   COUNTRY_221C   THER_AREA_96D7                          [BRAND_F649]   
10  COUNTRY_3536   THER_AREA_6CEE                          [BRAND_9259]   
11  COUNTRY_3536   THER_AREA_96D7                          [BRAND_056A]   
12  COUNTRY_3AD0   THER_A

In [20]:
train_data['launch_date'] = pd.to_datetime(train_data['launch_date'], errors='coerce')

price_unit_mean_pattern = (
    train_data.groupby(['country', 'therapeutic_area'])['price_unit']
    .mean()
    .reset_index()
    .rename(columns={'price_unit': 'mean_price_unit'})
)
print(price_unit_mean_pattern)

          country therapeutic_area  mean_price_unit
0    COUNTRY_0309   THER_AREA_032C         1.341232
1    COUNTRY_0309   THER_AREA_051D         1.202181
2    COUNTRY_0309   THER_AREA_644A         1.026566
3    COUNTRY_0309   THER_AREA_66C5         1.025165
4    COUNTRY_0309   THER_AREA_6CEE         4.436551
..            ...              ...              ...
367  COUNTRY_FA79   THER_AREA_66C5         1.079804
368  COUNTRY_FA79   THER_AREA_6CEE         3.771683
369  COUNTRY_FA79   THER_AREA_96D7         1.658719
370  COUNTRY_FA79   THER_AREA_980E         1.036424
371  COUNTRY_FA79   THER_AREA_CD59         1.075310

[372 rows x 3 columns]


In [22]:
train_data['month'] = train_data['launch_date'].dt.to_period('M')

price_unit_monthly_pattern = (
    train_data.groupby(['country', 'therapeutic_area', 'month'])['price_unit']
    .mean()
    .reset_index()
    .rename(columns={'price_unit': 'mean_price_unit'})
)

print(price_unit_monthly_pattern)

           country therapeutic_area    month  mean_price_unit
0     COUNTRY_0309   THER_AREA_032C  2019-10         1.236638
1     COUNTRY_0309   THER_AREA_032C  2020-02         1.457779
2     COUNTRY_0309   THER_AREA_051D  2019-11         1.202181
3     COUNTRY_0309   THER_AREA_644A  2017-02         1.009917
4     COUNTRY_0309   THER_AREA_644A  2017-03         1.028240
...            ...              ...      ...              ...
2229  COUNTRY_FA79   THER_AREA_980E  2017-10         1.007106
2230  COUNTRY_FA79   THER_AREA_980E  2018-11         1.267853
2231  COUNTRY_FA79   THER_AREA_CD59  2014-10         1.002022
2232  COUNTRY_FA79   THER_AREA_CD59  2015-12         1.219581
2233  COUNTRY_FA79   THER_AREA_CD59  2017-08         1.004327

[2234 rows x 4 columns]


In [23]:
train_data['month'] = train_data['launch_date'].dt.to_period('M')

price_unit_original = train_data[['country', 'therapeutic_area', 'month', 'price_unit']].sort_values(
    by=['therapeutic_area', 'month', 'country']
)

print(price_unit_original)

             country therapeutic_area    month  price_unit
11723   COUNTRY_53A5   THER_AREA_032C  2017-01    1.013681
12441   COUNTRY_53A5   THER_AREA_032C  2017-01    1.014095
13860   COUNTRY_53A5   THER_AREA_032C  2017-01    1.014322
14952   COUNTRY_53A5   THER_AREA_032C  2017-01    1.014116
15296   COUNTRY_53A5   THER_AREA_032C  2017-01    1.014214
...              ...              ...      ...         ...
117248  COUNTRY_9488   THER_AREA_CD59  2022-10    1.158955
117796  COUNTRY_9488   THER_AREA_CD59  2022-10    1.157120
115317  COUNTRY_C89B   THER_AREA_CD59  2022-10    1.084532
116507  COUNTRY_C89B   THER_AREA_CD59  2022-10    1.084532
118785  COUNTRY_C89B   THER_AREA_CD59  2022-10    1.084532

[118917 rows x 4 columns]
